In [ ]:
%matplotlib inline

# Parallel Programming

This is an elementary treatment that covers simple use cases that often arise in statistics and data science, and is focused mostly on parallelism on a multi-core computer. More specialized frameworks for parallel programming such as those listed are not covered:

- Message Passing Interface (MPI). Framework for parallelism based on message passing on a compute cluster. Often used when parallel processes need to frequently communicate with each other. More common in scientific than data analysis applications.
- GPU computing (CUDA, OpenCL). Framework for exploiting the massively parallel but basic compute units available in modern GPUs. Ideal when you need a large number of simple arithmetic function calls - for example, in deep learning. The [CuPy](https://cupy.chainer.org) library looks like a promising GPU-accelerated alternative to `numpy` if you have a modern NVidia GPU.
- Distributed computing (Spark, Dask). Framework for master-worker parallelism on a cluster of commodity machines. Master builds a graph of task dependencies and schedules to execute tasks in the appropriate order.

This first lecture is for mainly illustrating concepts. The second will provide more examples of using parallelism in Python programs.

Checking machine capabilities on a Mac

In [ ]:
! sysctl hw.physicalcpu hw.logicalcpu

For a Unix system, use this instead

In [ ]:
! lscpu --all --extended

**Referneces**

- [`threading` — Thread-based parallelism](https://docs.python.org/3.8/library/threading.html)
- [`multiprocessing` - Process-based “threading” interface](https://docs.python.org/3.8/library/multiprocessing.html)
- [`multiprocess` - a "better" `multiprocessing`](https://github.com/uqfoundation/multiprocess)
- [concurrent.futures — Launching parallel tasks](https://docs.python.org/3/library/concurrent.futures.html#module-concurrent.futures)
- [Concurrency with Processes, Threads, and Coroutines](https://pymotw.com/3//concurrency.html)

## Common sense

- Make it right before trying to make it fast
- Weight the trade-off between programmer and program time
- Don't blindly optimize - profile code before optimization
- A new algorithm or data structure can reduce the complexity class; parallelism generally only offers linear speed ups

## Basic Concepts

### Concurrent, parallel, distributed

- Concurrent programs have tasks with overlapping lifetimes, and concurrent programming is based on units of execution that can be run in any order with the same final result. Concurrent programs can but need not be executed in parallel - for example, the time-slicing performed by the Python's Global Interpreter Lock (GIL), which only allows one thread to be physically executed at any point in time, is an example of concurrency.
- Parallel programs run at the same time, for example, on multiple cores. If there is no need to control access to shared resources, these are known as *embarrassingly parallel* programs. All parallel programs are thus concurrent, but not vice versa.
- Distributed programs run over multiple computers. The term is most often associated with "Big Data" problems, where data transfer is the computing bottleneck - hence the philosophy of "bring the computation to the data".

### Synchronous and asynchronous calls

When there is concurrent access to a shared resource, errors can arise. For example, consider the following code executed by two concurrent processes which share the variable `i` with an initial value of 0:

```python
 i = i + 1
```

The following may happen: Process 1 reads (`i = 0`), Process 2 reads (`i = 0)`, Process 1 updates (`i = 1`), Process 2 overwrites the value put in by Process 1 (`i = 1`). Hence, after two updates, the value of `i` is 1 and not 2.

To avoid this problem, the first process to access the resource may *lock* it for exclusive access:

Process 1 reads and locks (`i = 0`), Process 2 tries to access `i` but it is locked, Process 1 updates and releases lock (`i = 1`), Process 2 now reads and locks (`i = 1`), and updates and unlocks (`i = 2`). Hence, after two updates, the value of `i` is 2 as it should be. Note that because the processes are *synchronous*, each process *blocks* access to the shared resource, forcing other process to wait - in Python, this waiting manifests as lack of access to the interpreter until the computation is complete.  In this context, *synchronous* and *blocking* have the same meaning.

*Asynchronous* or *non-blocking* calls were designed to allow access to the interpreter at all times - an asynchronous function call returns control to the interpreter immediately, and returns a *future* object that can be used to check at a later point in time if the computation is completed and if so, to return the answer.

### Concurrency is difficult

Designing programs that share resources or have other dependencies to run concurrently in an effective way is challenging. One of the main reasons for the increasing popularity of functional programming is that concurrency is generally easier with a functional programming style - for example, the famous map-reduce framework for distributed computing uses the `map` and `reduce` abstractions of functional programming.

Here we illustrate two famous problems that may arise in concurrent programs:

#### Race condition 

A *race condition* is one where the result depends on the order of accessing a shared resource, as in the simple example of updating `i` given above.

In the example below, up to 4 processes may be trying to increment and assign a new value to val at the same time. Because this takes two steps (increment the RHS, assign to LHS), it can happen that two or more processes increment at the same time, but this is only assigned and counted once.

In [ ]:
from multiprocessing import Pool, Value

How the code works will become clear in the next lecture, but the basic idea is that we have 4 processes trying to increment a value, and because of race conditions, this gives the wrong final count.

In [ ]:
def count_with_value(i):
    val.value += 1
    
val = Value('i', 0)
with Pool(processes=4) as pool:
    pool.map(count_with_value, range(1000))

val.value

##### Preventing race conditions by locking resources

In [ ]:
from multiprocessing import Lock

In [ ]:
lock = Lock()

def count_with_lock(i, lock=lock):
    lock.acquire()
    val.value += 1
    lock.release()

val = Value('i', 0)
with Pool(processes=4) as pool:
    pool.map(count_with_lock, range(1000))

val.value

##### Preventing race conditions by duplicating resources

It is usually easier and faster to make copies of resources for each process so that no sharing is required.

In [ ]:
import os
from multiprocessing import Array
import multiprocessing as mp

In [ ]:
def count_with_array(i):
    ix = mp.current_process().ident % 4
    arr[ix] += 1
    
arr = Array('i', [0]*4)

with Pool(processes=4) as pool:
    pool.map(count_with_array, range(1000))

arr[:], sum(arr)

#### Deadlock

Suppose each process needs both resources `i` and `j` to compute and return a value. Process 1 reads and locks `i` but before it can read `j`, Process 2 reads and locks `j`. Now both processes are waiting but cannot access the other shared resource, and hence the program hangs forever.

There is a story about the dining philosophers to help you remember this - a group of philosophers are seated on a round table - on the left of each philosopher is a fork and on the right is a knife. Philosophers need a fork and a knife to eat. At the beginning, each philosopher grabs the utensil to the right - now none of the philosophers have both a knife and a fork, and they starve to death.

To break deadlock, a possible strategy is to release locked resources upon failure and wait a random amount of time before trying to acquire it again.

### Amdahl's and Gustafson's laws

Suppose you had a million core machine. How much can you speed up your program by parallelizing it?

Ahmdahl recognized that the degree of speed-up is driven by the ratio of serial to parallelziable code segments. Serial parts are those where the order is essential $a \to b \to c$. Suppose the serial section of code takes up 10% of the the total run-time. Then the maximum speed up 10-fold - the serial part takes up 10% of the time, and you throw one million minus one cores at the parallelizable parts driving their execution time towards zero, giving the 10-fold speed up.

Gustafson pointed out that as the size of the problem increases, the time spent in  serial code segments typically goes down. For example, MCMC is inherently *all* serial, but for very large data sets, within each MCMC step, there is typically a need to calculate the likelihood of each data point, and this can be parallelized. We used this idea to achieve over two orders of magnitude speed up for fitting statistical mixture models using GPU computing.

### Opportunities for parallelism

There are 3 common areas where parallelism can be useful in an algorithm:

#### Data

- Decomposition of arrays along rows, columns, blocks
- Decomposition of trees into sub-trees

#### Tasks

- Loops
- Function calls

#### Pipelines

If there are a sequence of dependent stages, as soon as one stage completes and hands off to the next stage, it is ready to accept a job from the previous stage

- Data processing pipeline

$$
a \\
a \to b \\
a \to b \to c \\
a \to b \to c \\
a \to b \to c \\
b \to c \\
c
$$

### Common parallelization idioms

- Loop parallelism automatically performs the tasks in a loop in parallel
- Fork-join repeatedly splits execution into multiple branches (fork), then merging the branches (join), and is often used for recursive problems.
- Single program multiple data (SPMD), where the same program runs on multiple cores on one or more computers, but process different inputs.
- Master worker splits processes into a master that generates subproblems for multiple workers to complete 

#### Loop parallelism with `joblib`

In [ ]:
from joblib import Parallel, delayed
from time import sleep

In [ ]:
def func(n):
    sleep(n)
    return n

In [ ]:
%%time

[func(1) for i in range(4)]

The `joblib` library has a list comprehension idiom for loop parallelism. 

In [ ]:
%%time

Parallel(n_jobs=4)(delayed(func)(1) for i in range(4))

Note that the `delayed` function is just a technical device to allow function calls with the usual function call syntax.

In [ ]:
def func_args(n, *args, **kwargs):
    sleep(n)
    return args, kwargs

In [ ]:
%%time

Parallel(n_jobs=4)(delayed(func_args)(1, 2, 3, a=4, b=5) for i in range(4))

In the next lecture we will see the use of parallel ranges for loop parallelism with `nuumba` and `cython`

### Threads and processes

We can think of threads and processes as independent workers. The main difference is that threads run in the same memory space, while each process has its own memory space. This means that threads are "light-weight" compared to processes - they are faster to create and consume fewer resources.

However, because threads run in the same memory space, concurrency issues such as race conditions and deadlock can occur. Because of this, the most commonly used Python interpreter (CPython) uses the [Global Interpreter Lock (GIL)](http://www.dabeaz.com/python/UnderstandingGIL.pdf) to ensure that only a *single* thread is operating in a Python program any one time, and switches between multiple threads to give the illusion of parallelism. The take-home message is that you can use threads for parallel tasks that are are slow because of latency (e.g. network request), but should use processes for parallel tasks that are compute-intensive (e.g. mathematical calculations).

In [ ]:
def func_with_latency(n):
    sleep(n)
    return n

In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

Threads work well for high latency processes - e.g. disk access, network requests, await user input etc - here simulated with `sleep`

In [ ]:
%%time

with ProcessPoolExecutor(max_workers=4) as pool:
    pool.map(func_with_latency, [1,1,1,1])

In [ ]:
%%time

with ThreadPoolExecutor(max_workers=4) as pool:
    pool.map(func_with_latency, [1,1,1,1])

But not so well for compute-intensive processes

In [ ]:
import math

In [ ]:
def func_compute_intesnive(n):
    s = 0
    for i in range(1, n+1):
        s += math.exp(math.log(math.sqrt(math.pow(i, 2.0))))
    return s

In [ ]:
n = 1000000

In [ ]:
%%time

func_compute_intesnive(n)

In [ ]:
%%time

with ProcessPoolExecutor(max_workers=4) as pool:
    pool.map(func_compute_intesnive, [n,n,n,n])

In [ ]:
%%time

with ThreadPoolExecutor(max_workers=4) as pool:
    pool.map(func_compute_intesnive, [n,n,n,n])

## Embarrassingly parallel programs

Many statistical problems can be easily decomposed into independent tasks or data sets. Here are several examples:

- Monte Carlo integration
- Multiple chains of MCMC
- Bootstrap for confidence intervals
- Power calculations by simulation
- Permutation-resampling tests 
- Fitting same model on multiple data sets

These "low hanging fruits" are great because they offer a path to easy parallelism with minimal complexity.

In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing as mp
from multiprocessing import Pool, Value, Array
import time

### Vanilla Python

Toy problem: Estimate $\pi$ by sampling points at random within a box circumscribing the unit circle and counting the fraction that fall within the circle. This is a simple example of a Monte Carlo algorithm. We will parallelize this embarrassingly parallel problem.

In [ ]:
import numpy as np

In [ ]:
def mc_pi(n):
    s = 0
    for i in range(n):
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        if (x**2 + y**2) < 1:
            s += 1
    return 4*s/n

In [ ]:
%%time

mc_pi(int(1e5))

In [ ]:
%%time

res = [mc_pi(int(1e5)) for i in range(10)]

### The `concurrent.futures` module

Concurrent processes are processes that will return the same results regardless of the order in which they were executed. A "future" is something that will return a result sometime in the future.  The `concurrent.futures` module provides an event handler, which can be fed functions to be scheduled for future execution. This provides us with a simple model for parallel execution on a multi-core machine.

#### Using processes in parallel with `ProcessPoolExecutor`

We get a linear speedup as expected.

In [ ]:
%%time

with ProcessPoolExecutor(max_workers=4) as pool:
    res = pool.map(mc_pi, [int(1e5) for i in range(10)])

In [ ]:
np.array(list(res))

#### When you have many jobs

The `futures` object gives fine control over the process, such as adding callbacks and canceling a submitted job, but is computationally expensive. We can use the `chunksize` argument to reduce this cost when submitting many jobs - this specifies the number of tasks to be given to a worker at a time. A detailed explanation of `chunksize` is provided [here](https://stackoverflow.com/questions/53751050/python-multiprocessing-understanding-logic-behind-chunksize)

##### Using default `chunksize `

The total amount of computation whether you have 10 jobs of size 10,000,000 or 10,000 jobs of size 10,000 is essentially the same, so we would expect them both to take about the same amount of time, but this is not true due to the overhead described above.

In [ ]:
%%time

with ProcessPoolExecutor(max_workers=4) as pool:
    res = pool.map(mc_pi, [int(1e2) for i in range(int(1e4))])

##### Using `chunksize` of 100

In [ ]:
%%time

with ProcessPoolExecutor(max_workers=4) as pool:
    res = pool.map(mc_pi, [int(1e2) for i in range(int(1e4))], chunksize=100)

#### Fine control of processes

##### Status of processes

In [ ]:
def f1(x):
    return x**2

def f2(x, y):
    return x*y

In [ ]:
with ProcessPoolExecutor(max_workers=4) as pool:
    a = pool.submit(f2, 1, 1)
    b = pool.submit(f2, 1,2)
    c = pool.submit(f1, 10)    

    print('a running:', a.running())
    print('a done:', a.done())

    print('b running:', b.running())
    print('b done:', b.done())

    print('c running:', c.running())
    print('c done:', c.done())

    print('a result', a.result())
    print('b result', b.result())
    print('c result', c.result())

#### Canceling jobs and adding callbacks

For example, if you launch multiple versions of the same task for safety, you might want to cancel the duplicate tasks once one of them has completed.

Callbacks are to allow the function to notify you when some event occurs.

In [ ]:
njobs = 24

res = []

with ProcessPoolExecutor(max_workers=4) as pool:

    for i in range(njobs):
        res.append(pool.submit(f2, *np.random.rand(2)))
        if i % 2 == 0:
            res[i].add_done_callback(lambda future: print("Process done!"))
    res[4].cancel()
    if res[4].cancelled():
        print("Process 4 cancelled")

    for i, x in enumerate(res):
        while x.running():
            print("Running")
            time.sleep(1)
        if not x.cancelled():
            print(x.result())

#### Functions with multiple arguments

Using a pool and the `map` method with functions requiring multiple arguments can be done in two ways.

In [ ]:
def f(a, b):
    return a + b

In [ ]:
##### Using a function adapter

In [ ]:
def f_(args):
    return f(*args)

In [ ]:
xs = np.arange(24)
chunks = np.array_split(xs, xs.shape[0]//2)

In [ ]:
chunks

In [ ]:
with ProcessPoolExecutor(max_workers=4) as pool:
    res = pool.map(f_, chunks)
list(res)

##### Using multiple argument iterables

In [ ]:
with ProcessPoolExecutor(max_workers=4) as pool:
    res = pool.map(f, range(0,24,2), range(1,24,2))
list(res)

### Using processes in parallel with ThreadPoolExecutor

We do not get any speedup because the GIL only allows one thread to run at one time.

In [ ]:
%%time

with ThreadPoolExecutor(max_workers=4) as pool:
    res = pool.map(mc_pi, [int(1e5) for i in range(10)])

In [ ]:
np.array(list(res))

### Using `multiprocessing`

The `concurrent.futures.ProcessPoolExecutor` is actually a wrapper for `multiprocessing.Pool` to unify the threading and process interfaces. I typically just work directly with `mutliprocessing` since I don't have much use for threads. One nice thing about using `multiprocessing` apart from more fine-grai control if you need it, is that it typically works equally well for small numbers of large jobs, or large numbers of small jobs out of the box using a heuristic to guess the optimal `chunksize`.

In [ ]:
%%time

with mp.Pool(processes=4) as pool:
    res = pool.map(mc_pi, [int(1e5) for i in range(10)])

In [ ]:
np.array(res)

In [ ]:
%%time

with mp.Pool(processes=4) as pool:
    res = pool.map(mc_pi, [int(1e2) for i in range(int(1e4))])

In [ ]:
np.array(res)

#### Functions with multiple arguments

Multiprocessing `Pool` has a `starmap` method that removes the need to write a wrapper function.

In [ ]:
def f(a, b):
    return a + b

In [ ]:
xs = np.arange(24)
with Pool(processes=4) as pool:
    res = pool.starmap(f, np.array_split(xs, xs.shape[0]//2))
list(res)

##### Partial application

Sometimes, `functools.partial` can be used to reduce the number of arguments needed to just one.

In [ ]:
def f(a, b):
    return a * b

In [ ]:
from functools import partial

fp = partial(f, b=2)

In [ ]:
xs = np.arange(24)
with Pool(processes=4) as pool:
    res = pool.map(fp, xs)
np.array(list(res))

#### Blocking and non-blocking calls

In [ ]:
def func(n):
    time.sleep(n)
    return n

In [ ]:
with Pool(processes=4) as pool:
    res = pool.map(func, [3,3,3,3,3])
    print("Control back!")
res

In [ ]:
with Pool(processes=4) as pool:
    res = pool.map_async(func, [3,3,3,3,3])
    print("Control back!")
    print(res.ready())
    res.wait()
    print(res.ready())
    print(res.get())

#### Different jobs to different processes


In [ ]:
def f1(n):
    time.sleep(1)
    return n

def f2(n):
    time.sleep(1)
    return n**2

def f3(n):
    time.sleep(1)
    return n**3

def f4(n):
    time.sleep(1)
    return n**4

In [ ]:
%%time

with Pool(processes=4) as pool:
    res = []
    for i, f in enumerate([f4, f2, f3, f1]):
        res.append((i, pool.apply(f, [2])))
    print(res)

In [ ]:
%%time

with Pool(processes=4) as pool:
    res = []
    for i, f in enumerate([f4, f2, f3, f1]):
        res.append((i, pool.apply_async(f, [2])))
    print([(i, r.get()) for i, r in res])

#### Creating individual processes

If you need more control over individual processes than Pool provides - namely, if you need to share information across processes, you can work with individual workers and thread-safe memory structures. This is just for completeness as most data processing tasks do not require this level of control.

In [ ]:
def f(i):
    time.sleep(np.random.random())
    print(os.getpid(), i)

In [ ]:
for i in range(10):
    p = mp.Process(target=f, args=(i,))
    p.start()
    p.join()

#### Using Queues to share information between processes.

In [ ]:
def f1(q, i):
    time.sleep(np.random.random())
    q.put((os.getpid(), i))

In [ ]:
q = mp.Queue()

res = []
for i in range(10):
    p = mp.Process(target=f1, args=(q,i,))
    p.start()
    res.append(q.get())
    p.join()

res

#### Using Value and Array for sharing data

#### Counting number of jobs (1)

This does not work.

In [ ]:
def f2(i):
    global counter
    counter = counter + 1
    print(os.getpid(), i)

##### Checking

In [ ]:
counter = 0
f2(10)
print(counter)

In [ ]:
counter = 0

for i in range(10):
    p = mp.Process(target=f2, args=(i,))
    p.start()
    p.join()

##### Note that separate processes have their own memory and DO NOT share global memory

In [ ]:
counter

#### Counting number of jobs (2)

We can use shared memory to do this, but it is slow because multiprocessing has to ensure that only one process gets to use counter at any one time. Multiprocesing provides Value and Array shared memory variables, but you can also convert arbitrary Python variables into shared memory objects (less efficient).

In [ ]:
def f3(i, counter, store):
    counter.value += 1
    store[os.getpid() % 10] += 1

In [ ]:
%%time

counter = mp.Value('i', 0)
store = mp.Array('i', [0]*10)

for i in range(int(1e2)):
    p = mp.Process(target=f3, args=(i, counter, store))
    p.start()
    p.join()

print(counter.value)
print(store[:])

#### Avoiding use of shared memory

##### Counting number of jobs (3)

We should try to avoid using shared memory as much as possible in parallel jobs as they drastically reduce efficiency. One useful approach is to use the `map-reduce` pattern. We should also use Pool to reuse processes rather than spawn too many of them. 

In [ ]:
def f4(i):
    return (os.getpid(), 1, i)

In [ ]:
%%time

# map step
with mp.Pool(processes=4) as pool:
    res = pool.map(f4, range(int(1e2)))

##### Reduce steps

In [ ]:
res = np.array(res)

In [ ]:
res[np.random.choice(len(res), 10)]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(res, columns=['pid', 'one', 'i'])

In [ ]:
df.groupby('pid').sum()